### NAME : RAJASREE S
### ROLL NO : 215229147

## Lab-12. Image corpus creation and Transfer Learning in CNN

1. Import libraries

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import random
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import pandas as pd

2. Initialize some parameters

In [2]:
now=datetime.datetime.now

batch_size=128
num_classes=5
epochs=5

img_rows,img_cols=28,28
filters=32
pool_size=2
kernel_size=3

3. Partition MNIST dataset

In [3]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train_lt5 = x_train[y_train<5]
y_train_lt5 = y_train[y_train<5]
x_test_lt5 = x_test[y_test<5]
y_test_lt5 = y_test[y_test<5]

x_train_gte5 = x_train[y_train>=5]
y_train_gte5 = y_train[y_train>=5]-5
x_test_gte5 = x_test[y_test>=5]
y_test_gte5 = y_test[y_test>=5]-5

11501568/11490434 [==============================] - 0s 0us/step


4. Define the "feature" layers

In [4]:
feature_layer=[Conv2D(filters=32, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(28,28,1)),
               Conv2D(filters=32, kernel_size=kernel_size, activation='relu', input_shape=(28,28,1)),
               MaxPooling2D(pool_size),
               Dropout(rate=0.25),
               Flatten()]

5. Define the "classification" layers

In [5]:
classification_layer=[Dense(batch_size, activation='relu'),
                      Dropout(rate=0.5),
                      Dense(num_classes, activation='softmax')]

In [6]:
model = Sequential(feature_layer + classification_layer)

6. Define a Sequential model

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

7. Create a function train model()

In [8]:
def train_model(model,train,test,num_classes):
  train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
  test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)
  train[0] = train[0].astype('float32')
  test[0] = test[0].astype('float32')
  train[0] /= 255
  test[0] /= 255
  print(train[0].shape)
  print(test[0].shape)
  s = pd.Series(train[1])
  train[1] = pd.get_dummies(s)
  train[1] = train[1].values
  s = pd.Series(test[1])
  test[1] = pd.get_dummies(s)
  test[1] = test[1].values

  model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])
  train_start_time=now()
  model.fit(train[0],train[1],epochs=epochs,verbose=2)
  print('\n')
  print('\n')
  print('Training time: %s' % (now() - train_start_time))
  score=model.evaluate(test[0],test[1],verbose=0)
  print('test loss ',score[0])
  print('test accuracy ',score[1])

8. Training on digits 5 to 9

In [9]:
train_model(model,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 61s - loss: 1.5712 - accuracy: 0.3154 - 61s/epoch - 67ms/step
Epoch 2/5
919/919 - 58s - loss: 1.5026 - accuracy: 0.4682 - 58s/epoch - 63ms/step
Epoch 3/5
919/919 - 58s - loss: 1.4138 - accuracy: 0.5803 - 58s/epoch - 63ms/step
Epoch 4/5
919/919 - 60s - loss: 1.3013 - accuracy: 0.6555 - 60s/epoch - 65ms/step
Epoch 5/5
919/919 - 58s - loss: 1.1694 - accuracy: 0.7028 - 58s/epoch - 64ms/step




Training time: 0:04:56.020839
test loss  1.0368167161941528
test accuracy  0.8463279008865356


9. Freeze Feature Layers

In [10]:
for l in feature_layer:
  l.trainable=False

10. Print Summary

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

11. Training for digits 0 to 4 based on digits 5 to 9

In [12]:
train_model(model,[x_train_lt5, y_train_lt5],[x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 20s - loss: 1.4469 - accuracy: 0.4072 - 20s/epoch - 21ms/step
Epoch 2/5
957/957 - 18s - loss: 1.2458 - accuracy: 0.5408 - 18s/epoch - 19ms/step
Epoch 3/5
957/957 - 18s - loss: 1.0890 - accuracy: 0.6669 - 18s/epoch - 19ms/step
Epoch 4/5
957/957 - 17s - loss: 0.9545 - accuracy: 0.7621 - 17s/epoch - 18ms/step
Epoch 5/5
957/957 - 18s - loss: 0.8447 - accuracy: 0.8108 - 18s/epoch - 18ms/step




Training time: 0:01:31.386530
test loss  0.7216097116470337
test accuracy  0.8838295340538025


12. Reversing the training process

In [13]:
INPUT_SHAPE = (img_rows,img_cols,1)
feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',padding="same",input_shape=(28,28,1)),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=(28,28,1)),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()]
classification_layers = [Dense(128,activation='relu'),Dropout(0.5),

Dense(5,activation='softmax')]

In [14]:
modelReverse = Sequential(feature_layers+classification_layers)
modelReverse.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                      

In [15]:
train_model(model,[x_train_lt5, y_train_lt5],[x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 18s - loss: 0.7529 - accuracy: 0.8419 - 18s/epoch - 19ms/step
Epoch 2/5
957/957 - 18s - loss: 0.6798 - accuracy: 0.8579 - 18s/epoch - 19ms/step
Epoch 3/5
957/957 - 20s - loss: 0.6207 - accuracy: 0.8716 - 20s/epoch - 20ms/step
Epoch 4/5
957/957 - 18s - loss: 0.5720 - accuracy: 0.8814 - 18s/epoch - 19ms/step
Epoch 5/5
957/957 - 19s - loss: 0.5303 - accuracy: 0.8876 - 19s/epoch - 19ms/step




Training time: 0:01:33.381165
test loss  0.43036308884620667
test accuracy  0.933060884475708


In [16]:
for l in feature_layers:
  l.trainable=False

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

In [18]:
train_model(model,[x_train_gte5, y_train_gte5],[x_test_gte5, y_test_gte5], num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 18s - loss: 1.3622 - accuracy: 0.4682 - 18s/epoch - 20ms/step
Epoch 2/5
919/919 - 18s - loss: 1.1920 - accuracy: 0.5464 - 18s/epoch - 20ms/step
Epoch 3/5
919/919 - 18s - loss: 1.0654 - accuracy: 0.6120 - 18s/epoch - 19ms/step
Epoch 4/5
919/919 - 18s - loss: 0.9698 - accuracy: 0.6678 - 18s/epoch - 19ms/step
Epoch 5/5
919/919 - 18s - loss: 0.8882 - accuracy: 0.7112 - 18s/epoch - 19ms/step




Training time: 0:01:29.946745
test loss  0.7727604508399963
test accuracy  0.7821435928344727


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        